# Семантичне Ядро

У цьому прикладі коду ви використовуватимете [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel), AI-фреймворк, щоб створити базового агента.

Мета цього прикладу — показати вам кроки, які ми пізніше використовуватимемо в додаткових прикладах коду під час реалізації різних агентних шаблонів.


## Імпорт необхідних пакетів Python


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Створення клієнта

У цьому прикладі ми будемо використовувати [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) для доступу до LLM.

`ai_model_id` визначено як `gpt-4o-mini`. Спробуйте змінити модель на іншу, доступну на ринку GitHub Models, щоб побачити різні результати.

Щоб використовувати `Azure Inference SDK`, який застосовується для `base_url` у GitHub Models, ми скористаємося конектором `OpenAIChatCompletion` у Semantic Kernel. Також є інші [доступні конектори](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion), які дозволяють використовувати Semantic Kernel для інших постачальників моделей.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Створення агента

Нижче ми створюємо агента під назвою `TravelAgent`.

У цьому прикладі ми використовуємо дуже прості інструкції. Ви можете змінити ці інструкції, щоб побачити, як агент реагує по-іншому.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Запуск агента

Тепер ми можемо запустити агента, визначивши потік типу `ChatHistoryAgentThread`. Усі необхідні системні повідомлення передаються агенту через аргумент `messages` ключового слова invoke_stream.

Після того, як ці параметри визначені, ми створюємо `user_inputs`, які будуть тим, що користувач надсилає агенту. У цьому випадку ми встановили це повідомлення як `Plan me a sunny vacation`.

Не соромтеся змінювати це повідомлення, щоб побачити, як агент реагує по-різному.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**Відмова від відповідальності**:  
Цей документ був перекладений за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, будь ласка, майте на увазі, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ на його рідній мові слід вважати авторитетним джерелом. Для критичної інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникають внаслідок використання цього перекладу.
